In [90]:
import pandas as pd
import datetime
import numpy as np
from datetime import timedelta
import seaborn as sns
from matplotlib.pyplot import figure
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier

In [16]:
user=pd.read_csv('/Users/renfeigao/Desktop/take home book/relax_challenge/takehome_users.csv',
                 encoding = "ISO-8859-1")
user_engagement=pd.read_csv('/Users/renfeigao/Desktop/take home book/relax_challenge/takehome_user_engagement.csv',
                           encoding = "ISO-8859-1")

In [18]:
user.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,4/22/14 3:53,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,11/15/13 3:45,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,3/19/13 23:14,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,5/21/13 8:09,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,1/17/13 10:14,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [19]:
user_engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


#### Flag Adoption

In [30]:
user_engagement.time_stamp=pd.to_datetime(user_engagement.time_stamp)
user_engagement=user_engagement.sort_values(by=['user_id','time_stamp'])
user_engagement['today']=pd.to_datetime('today')
user_engagement['today']=user_engagement['today'].dt.date
user_engagement['time_stamp']=user_engagement['time_stamp'].dt.date
user_engagement['days']=(user_engagement['today']-user_engagement['time_stamp'])/np.timedelta64(1, 'D')

In [40]:
user_engagement['dif']=user_engagement.groupby('user_id').days.diff(periods=2)
user_engagement.loc[user_engagement['dif']>-7,'adopted']=1
user_engagement_f=user_engagement.groupby('user_id').adopted.sum().reset_index(name='adopt_sum')
user_engagement_f['adopt']=0
user_engagement_f.loc[user_engagement_f.adopt_sum>0,'adopt']=1

#### Create features to build model

In [67]:
user_f=user.merge(user_engagement_f,left_on='object_id',right_on='user_id')

In [69]:
user_f=user_f.drop(columns=['adopt_sum','user_id'])

In [73]:
user_f['today']=pd.to_datetime('today')
user_f['today']=user_f['today'].dt.date
user_f['creation_time']=pd.to_datetime(user_f['creation_time'])
user_f['createion_date']=user_f['creation_time'].dt.date
user_f['create_age']=(user_f['today']-user_f['createion_date'])/np.timedelta64(1, 'D')
user_f['hour']=user_f['creation_time'].apply(lambda x: x.hour)
user_f['weekday']=user_f['creation_time'].dt.day_name()

In [82]:
user_f['email_com']=user_f['email'].apply(lambda x:x.split("@")[1])
user_f.loc[user_f['invited_by_user_id'].isnull(),'invited_by_user']=-1
user_f['invited_by_user']=user_f['invited_by_user']+1
user_f['invited_by_user_id']=user_f['invited_by_user_id'].fillna(-1)
user_f['invited_by_user']=user_f['invited_by_user'].fillna(0)

In [85]:
user_f=user_f.drop(columns=['object_id','creation_time','name','email','today','createion_date'])

In [87]:
user_model=pd.get_dummies(user_f,drop_first=True)

In [91]:
Xtrain,Xtest,ytrain,ytest=train_test_split(user_model.drop(columns=['adopt']),user_model['adopt'],
                                           stratify=user_model['adopt'],test_size=0.33,random_state=42)

In [92]:
clf_xg = XGBClassifier(learning_rate =0.01, n_estimators=10000, max_depth=4, min_child_weight=1, subsample=0.8, colsample_bytree=0.8, 
                       objective= 'binary:logistic', 
            nthread=4, scale_pos_weight=2.5, seed=27, reg_lambda = 1.2)
clf_xg.fit(Xtrain, ytrain, eval_set=[(Xtrain, ytrain), (Xtest, ytest)], 
                eval_metric= 'auc', verbose= 100, early_stopping_rounds= 200)

[0]	validation_0-auc:0.935883	validation_1-auc:0.928779
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 200 rounds.
[100]	validation_0-auc:0.988693	validation_1-auc:0.977864
[200]	validation_0-auc:0.991203	validation_1-auc:0.98023
[300]	validation_0-auc:0.992208	validation_1-auc:0.981095
[400]	validation_0-auc:0.993756	validation_1-auc:0.981718
[500]	validation_0-auc:0.995463	validation_1-auc:0.982877
[600]	validation_0-auc:0.996239	validation_1-auc:0.983091
[700]	validation_0-auc:0.996771	validation_1-auc:0.983145
Stopping. Best iteration:
[584]	validation_0-auc:0.996161	validation_1-auc:0.98321



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=10000,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1.2, scale_pos_weight=2.5, seed=27,
       silent=True, subsample=0.8)

In [93]:
feature_importances = pd.DataFrame(clf_xg.feature_importances_,
                                   index = Xtrain.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances.head(10)

,importance
create_age,0.311852
last_session_creation_time,0.290605
org_id,0.109455
invited_by_user_id,0.076327
hour,0.073004
weekday_Sunday,0.012587
email_com_gmail.com,0.010875
creation_source_PERSONAL_PROJECTS,0.010170
opted_in_to_mailing_list,0.010069
creation_source_SIGNUP,0.009667


From the mdoeling, the create age and last_session_creation_time are really important feature for adoption. That make sence, the new creation user are more likely to be active user.